In [13]:
#import libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [14]:
#Configuração dos geradores de dados de treinamento e validação

train = ImageDataGenerator(rescale=1/255) #Normalizar os valores de pixels para o intervalo [0,1]
validation = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

In [15]:
#Carregar os dados de treinamento e validação

train_dataset = train.flow_from_directory('Dataset/training/',
                                         target_size = (200,200), #Redimensionar as imagens para 200x200 pixels
                                         batch_size = 3, #numero de pixels por lote
                                         class_mode = 'categorical') #Modo de classificação (categorica)

validation_dataset = train.flow_from_directory('Dataset/validation/',
                                         target_size = (200,200),
                                         batch_size = 3,
                                         class_mode = 'categorical')

test_dataset = test.flow_from_directory('Dataset/testing/',
                                         target_size=(200,200),
                                         batch_size=3,
                                         class_mode='categorical')

Found 3213 images belonging to 3 classes.
Found 1140 images belonging to 3 classes.
Found 1052 images belonging to 3 classes.


In [16]:
#Verificar os indices das classes

train_dataset.class_indices

{'clean': 0, 'litter': 1, 'recycle': 2}

In [17]:
# Definindo o modelo da rede neural (MLP)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(200, 200, 3)),  # Transforma os dados em um vetor
    tf.keras.layers.Dense(512, activation='relu'),  # Camada densa com ativação ReLU
    tf.keras.layers.Dropout(0.5),  # Adicionando Dropout para reduzir overfitting
    tf.keras.layers.BatchNormalization(),  # Adicionando normalização em lotes
    tf.keras.layers.Dense(256, activation='relu'),  # Camada densa com ativação ReLU
    tf.keras.layers.Dense(128, activation='relu'),  # Adicionando mais uma camada densa
    tf.keras.layers.Dense(3, activation='softmax')  # Camada de saída com ativação softmax (três classes)
])

In [18]:
# Compilando o modelo com uma taxa de aprendizado ajustada

model.compile(optimizer = RMSprop(learning_rate=0.0001), #Otimizador RMSprop com taxa de aprendizado de 0.001
             loss = 'binary_crossentropy', #Função de perda para classificação multiclasse
             metrics=['accuracy']) #Métrica para avaliação do modelo

In [19]:
# Treinando o modelo

history = model.fit(train_dataset,
          steps_per_epoch=3,  # Número total de lotes (batches) por época
          epochs=50,  # Número de épocas de treinamento
          validation_data=validation_dataset,  # Dados de validação
          validation_steps=3)  # Número total de lotes (batches) de validação

Epoch 1/50
3/3 [==============================] - 6s 1s/step - loss: 0.6667 - accuracy: 0.6667 - val_loss: 1.2261 - val_accuracy: 0.2222
Epoch 2/50
3/3 [==============================] - 3s 1s/step - loss: 0.7597 - accuracy: 0.1111 - val_loss: 1.5893 - val_accuracy: 0.1111
Epoch 3/50
3/3 [==============================] - 2s 842ms/step - loss: 0.6293 - accuracy: 0.5556 - val_loss: 1.0285 - val_accuracy: 0.2222
Epoch 4/50
3/3 [==============================] - 3s 1s/step - loss: 0.6617 - accuracy: 0.4444 - val_loss: 0.6695 - val_accuracy: 0.3333
Epoch 5/50
3/3 [==============================] - 2s 836ms/step - loss: 0.7399 - accuracy: 0.3333 - val_loss: 0.8620 - val_accuracy: 0.4444
Epoch 6/50
3/3 [==============================] - 2s 827ms/step - loss: 0.7014 - accuracy: 0.3333 - val_loss: 1.1693 - val_accuracy: 0.1111
Epoch 7/50
3/3 [==============================] - 2s 708ms/step - loss: 0.6898 - accuracy: 0.2222 - val_loss: 0.6951 - val_accuracy: 0.4444
Epoch 8/50
3/3 [=============

In [20]:
# Avaliação no conjunto de teste

y_pred = model.predict(test_dataset)
y_true = test_dataset.classes

351/351 [==============================] - 48s 136ms/step


In [21]:
# Calculando a matriz de confusão

conf_matrix = confusion_matrix(y_true, y_pred.argmax(axis=1))

In [22]:
# Calculando outras métricas

accuracy = accuracy_score(y_true, y_pred.argmax(axis=1))
classification_rep = classification_report(y_true, y_pred.argmax(axis=1), zero_division=1)
roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')

In [23]:
# Exibindo os resultados
print("Matriz de Confusão:")
print(conf_matrix)

print("\nAccuracy:", accuracy)
print("\nClassification Report:")
print(classification_rep)
print("\nAUC-ROC:", roc_auc)

Matriz de Confusão:
[[300   2  58]
 [312   2  54]
 [280   3  41]]

Accuracy: 0.32604562737642584

Classification Report:
              precision    recall  f1-score   support

           0       0.34      0.83      0.48       360
           1       0.29      0.01      0.01       368
           2       0.27      0.13      0.17       324

    accuracy                           0.33      1052
   macro avg       0.30      0.32      0.22      1052
weighted avg       0.30      0.33      0.22      1052


AUC-ROC: 0.5043751755139869
